In [25]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget

import importlib
import os

from matplotlib import pyplot as plt
from sqlalchemy import create_engine

from ppcollapse import setup_logger
from ppcollapse.cvat_to_db import create_collapses_from_cvat
from ppcollapse.setup_django_ppcx import setup_django
from ppcollapse.utils.config import ConfigManager
from ppcollapse.utils.database import get_collapses_df, get_image

# Allow Django ORM to work in Jupyter's async environment
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Setup Django
config = ConfigManager(config_path="config.yaml")
setup_django(db_config=config.get("database"))

logger = setup_logger(level="INFO", name="ppcx")
config = ConfigManager(config_path="config.yaml")
db_engine = create_engine(config.db_url)

# Import Django models
ppcx_app_models = importlib.import_module("ppcx_app.models")
Collapse = ppcx_app_models.Collapse
Image = ppcx_app_models.Image

Django is already configured. Skipping setup_django().


Manual fix of some collapse dates


In [26]:
# 09/10/2017 - volume wrong (should be 3500 m3)

skip = True
if not skip:
    collapse_id = 1122
    collapse = Collapse.objects.get(id=collapse_id)
    collapse.volume = 3500
    collapse.save()

In [27]:
# Create new collapse from cvat data labels
skip = True
if not skip:
    created_ids = create_collapses_from_cvat(
        "data/job_23_annotations_2025_11_25_17_21_42_cvat for images 1.1.xml",
        label_name="collapse",
        image_search_mode="auto",
        time_tolerance_seconds=300,
        remove_reg_suffix=True,
        dry_run=True,
    )

In [34]:
# Filter all the day with more than one collapse to check for possible errors in the volume
df = get_collapses_df(db_engine=db_engine)
dates_with_multiple_collapses = df["date"].value_counts()
dates_with_multiple_collapses = dates_with_multiple_collapses[
    dates_with_multiple_collapses > 1
].index.tolist()
dates_with_multiple_collapses.sort()
for date in dates_with_multiple_collapses:
    collapses = Collapse.objects.filter(image__acquisition_timestamp__date=date)
    # Skip if none of the collapse has no volume assigned
    if all(c.volume is None for c in collapses):
        continue
    print(f"Collapses on {date}: {[c.id for c in collapses]}")

Collapses on 2014-09-12 00:00:00: [1044, 1045]
Collapses on 2017-10-09 00:00:00: [1122, 1503]
Collapses on 2018-09-19 00:00:00: [1140, 1141]
Collapses on 2018-09-27 00:00:00: [1146, 1147]
Collapses on 2019-07-15 00:00:00: [1174, 1175]
Collapses on 2019-09-01 00:00:00: [1196, 1197]
Collapses on 2019-09-25 00:00:00: [1206, 1207]
Collapses on 2020-06-06 00:00:00: [1214, 1215]
Collapses on 2020-06-27 00:00:00: [1224, 1223]
Collapses on 2020-07-04 00:00:00: [1226, 1227]
Collapses on 2020-07-05 00:00:00: [1228, 1229]
Collapses on 2020-07-13 00:00:00: [1233, 1234]
Collapses on 2020-07-21 00:00:00: [1237, 1238]
Collapses on 2020-07-26 00:00:00: [1240, 1241]
Collapses on 2020-07-27 00:00:00: [1243, 1242]
Collapses on 2020-07-31 00:00:00: [1247, 1246]
Collapses on 2020-08-08 00:00:00: [1252, 1253]
Collapses on 2020-08-11 00:00:00: [1255, 1256]
Collapses on 2020-08-15 00:00:00: [1260, 1261, 1262]
Collapses on 2020-09-15 00:00:00: [1278, 1279]


In [ ]:
# 2017-10-09
skip = True
if not skip:
    c1 = Collapse.objects.get(id=1122)
    c1.volume = 3500.0
    c2 = Collapse.objects.get(id=1503)
    c2.volume = 15000.0
    c1.save()
    c2.save()

In [ ]:
# Collapses on 2018-09-19 00:00:00: [1140, 1141] --> correct
# 2018/09/27 volume is assigned to the wrong collapse. Swap volumes
skip = True
if not skip:
    date = "2018-09-27"
    collapses = Collapse.objects.filter(image__acquisition_timestamp__date=date)
    print(f"Collapses on {date}: {[c.id for c in collapses]}")
    c1 = collapses[0]
    c2 = collapses[1]
    v1 = c1.volume
    v2 = c2.volume
    c1.volume = v2
    c2.volume = v1
    print(f"Swapping volumes: c1={c1.id} v={c1.volume}, c2={c2.id} v={c2.volume}")
    c1.save()
    c2.save()

In [ ]:
# Collapses on 2019-07-15 00:00:00: [1174, 1175] --> worng image assigned. but probably correct volumes.

# Collapses on 2019-09-01 00:00:00: [1196, 1197] --> correct

# Collapses on 2019-09-25 00:00:00: [1206, 1207] --> correct

In [ ]:
# Ignore all other 2020 data... redo mapping...

# 2020/06/27 volume is assigned to the wrong collapse
skip = True
if not skip:
    date = "2020-06-27"
    collapses = Collapse.objects.filter(image__acquisition_timestamp__date=date)
    print(f"Collapses on {date}: {[c.id for c in collapses]}")

    c1 = Collapse.objects.get(id=1223)
    c1.volume = 22565.0
    c2 = Collapse.objects.get(id=1224)
    c2.volume = None

    c1.save()
    c2.save()

## OLD CELL


In [ ]:
# List of collapse ID to delete
collapse_ids_to_delete = [
    49,
    50,
    56,
    57,
    61,
    62,
    67,
    71,
    77,
    79,
    81,
    83,
    84,
    91,
    93,
    94,
    101,
    104,
    113,
    117,
    118,
    119,
    120,
    121,
    122,
    123,
    125,
    126,
    129,
    130,
    133,
    137,
    139,
    141,
    145,
    147,
    148,
    149,
    150,
    160,
    168,
    169,
    171,
    173,
    174,
    177,
    178,
    182,
    188,
]

In [ ]:
engine = create_engine(config.db_url)
df = get_collapses_df(engine)
if df.empty:
    logger.warning("No collapses found in database.")
df

In [ ]:
from shapely.ops import unary_union

# Example: merge multiple polygons from your collapse dataframe

collapse_1 = df.iloc[0]
collapse_2 = df.iloc[1]

geom1 = shapely_wkt.loads(collapse_1["geom_wkt"])
geom2 = shapely_wkt.loads(collapse_2["geom_wkt"])

# Merge into single geometry (will be MultiPolygon if not contiguous)
merged_geom = unary_union([geom1, geom2])

# If they ARE contiguous, this will return a Polygon
# If they're NOT contiguous, this will return a MultiPolygon
print(type(merged_geom))  # Polygon or MultiPolygon


In [ ]:
image = get_image(image_id=int(collapse_1["image_id"]), config=config)
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(image)
# Handle both Polygon and MultiPolygon
if merged_geom.geom_type == "Polygon":
    # Single polygon - plot directly
    xs, ys = merged_geom.exterior.xy
    ax.plot(xs, ys, color="red", linewidth=2)
    ax.fill(xs, ys, facecolor="red", edgecolor="none", alpha=0.3)
elif merged_geom.geom_type == "MultiPolygon":
    # Multiple polygons - iterate and plot each
    for poly in merged_geom.geoms:
        xs, ys = poly.exterior.xy
        ax.plot(xs, ys, color="red", linewidth=2)
        ax.fill(xs, ys, facecolor="red", edgecolor="none", alpha=0.3)
else:
    raise ValueError("Merged geometry is neither Polygon nor MultiPolygon.")
ax.set_axis_off()
ax.set_title("Collapse Geometry", fontsize=10, pad=5)

In [ ]:
# Now import Django models
from ppcx_app.models import Collapse
from shapely import wkt as shapely_wkt


In [ ]:
# Get the merged geometry
merged_geom = unary_union([geom1, geom2])

# Convert Shapely geometry to Django GEOSGeometry
wkt_str = shapely_wkt.dumps(merged_geom)
django_geom = GEOSGeometry(wkt_str, srid=0)

collapse_obj = Collapse.objects.get(id=collapse_1["id"])
collapse_obj.geom = django_geom
